In [1]:
import yfinance as yf
import pandas as pd
import requests

In [2]:
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://en.wikipedia.org/"
}

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
r = requests.get(url, headers=headers)
spx = pd.read_html(r.text)[1]

/tmp/ipykernel_21113/2959688793.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  spx = pd.read_html(r.text)[1]


In [4]:
spx

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [6]:
import time

In [7]:
tickers = spx["Symbol"].str.replace(".", "-", regex=False).tolist()
tickers.append("^GSPC")
# 2) Params
START = "2015-01-01"   # or None
END = None             # e.g., "2025-10-11"
INTERVAL = "1d"        # "1d","1h","1wk","1mo"
BATCH = 100            # safe batch size for yfinance

# 3) Batch download
def batched(iterable, n):
    for i in range(0, len(iterable), n):
        yield iterable[i:i+n]

frames = []
for batch in batched(tickers, BATCH):
    df = yf.download(
        " ".join(batch),
        start=START,
        end=END,
        interval=INTERVAL,
        group_by="ticker",
        auto_adjust=False,
        threads=True,
        progress=False,
    )
    # Ensure MultiIndex columns [Ticker, Field]
    if isinstance(df.columns, pd.MultiIndex):
        frames.append(df)
    else:
        # Single ticker edge case -> promote to MultiIndex
        df.columns = pd.MultiIndex.from_product([batch, df.columns])
        frames.append(df)
    time.sleep(0.5)

# 4) Combine and optional reshape
wide = pd.concat(frames, axis=1).sort_index(axis=1)            # columns: (Ticker, Field)
long = (
    wide.stack(level=0)                                        # index -> add Ticker
        .rename_axis(index=["Date", "Ticker"])
        .reset_index()
)

# 5) Examples of saving
# wide.to_parquet("sp500_ohlcv_wide.parquet")
# long.to_parquet("sp500long.parquet")
# wide.to_csv("sp500_ohlcv_wide.csv")
long.to_csv("sp500long.csv")

# print(wide.columns.names)   # ['Ticker','Attributes'] expected
# print(wide.tail(3))

/tmp/ipykernel_21113/2478591040.py:38: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  wide.stack(level=0)                                        # index -> add Ticker
